In [ ]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

# Seeding
os.environ['PYTHONHASHSEED'] = '42'
np.random.seed(42)
tf.random.set_seed(42)

# Path
dataset_path = "D:/cv/newCvcp/dataset/nonaug/test"
save_path = "D:/cv/newCvcp/prediction/non-aug"
model_path = "D:/cv/newCvcp/New folder/unet-non-aug.keras"

# Create folder
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Folder to save predicted mask
create_dir(save_path)

# Load model
model = tf.keras.models.load_model(model_path)
model.summary()

test_x = sorted(glob(os.path.join(dataset_path, "images", "*")))
print(f"Test Images: {len(test_x)}")

time_taken = []

for x in tqdm(test_x):
    name = os.path.basename(x)  # Extract the filename from the path
    
    # Read the image using OpenCV with color channels
    img = cv2.imread(x, cv2.IMREAD_COLOR)
    
    # Normalize the image data
    img = img / 255.0
    
    # Add a batch dimension to the image for model prediction
    img = np.expand_dims(img, axis=0)
    
    start_time = time.time()  # Record the start time before making the prediction
    
    pred_mask = model.predict(img)  # Use the model to predict the mask for the input image
    
    total_time = time.time() - start_time  # Calculate the total time taken for prediction
    time_taken.append(total_time)  # Append the total time taken to the time_taken list
    
    # Threshold the predicted mask
    pred_mask = (pred_mask > 0.5).astype(np.uint8) * 255
    
    # Convert the predicted mask to the same shape as the original image
    pred_mask = cv2.resize(pred_mask[0], (img.shape[2], img.shape[1]), interpolation=cv2.INTER_NEAREST)
    
    # Overlay the mask on the original image for visualization
    overlay = cv2.addWeighted(img[0].astype(np.uint8), 0.7, cv2.cvtColor(pred_mask, cv2.COLOR_GRAY2BGR).astype(np.uint8), 0.3, 0)
    
    # Save the overlay image
    cv2.imwrite(os.path.join(save_path, f"{os.path.splitext(name)[0]}_segmented.jpg"), overlay)

# Calculate mean time per inference and mean FPS
mean_time = np.mean(time_taken)
mean_fps = 1 / mean_time

print(f"Mean Time: {mean_time:.5f} seconds")
print(f"Mean FPS: {mean_fps:.5f}")
